In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
from transformers import AutoModelForTokenClassification ,AutoTokenizer,DataCollatorForTokenClassification
from transformers import TrainingArguments,Trainer
import torch
import evaluate #pip install evaluate
import seqeval #pip install seqeval
from datasets import load_dataset

2025-06-24 14:00:49.043619: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750773649.551689      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750773649.679805      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


ModuleNotFoundError: No module named 'evaluate'

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('google-bert/bert-base-chinese')

In [ ]:
ds = load_dataset('doushabao4766/msra_ner_k_V3')

In [ ]:
ds

In [ ]:
ds['train']['tokens']

for item in ds['train']:
    print(item['tokens'])
    print(item['ner_tags'])
    break

###实体映射字典

‘O’:0</br>
'B-PER':1</br>
'I-PER':2</br>
'B-LOC':3</br>
'I-LOC':4</br>
'B-ORG':5</br>
'I-ORG':6</br>

获取所有tag标签

In [ ]:
tags_id =set()
for items in ds['train']:
    tags_id.update(items['ner_tags'])

tags_id

In [ ]:
entites=['O']+list({'PER','LOC','ORG'})
tags =['O']

for entity in entites[1:]:
    tags.append('B-'+entity.upper())
    tags.append('I-'+entity.upper())

entity_index={entity:i for i,entity in enumerate(entites)}

In [ ]:
entity_index

In [ ]:
tags

In [ ]:
def data_input_proc(item):     
    input_data = tokenizer(item['tokens'],#已经分词的文本
                            truncation=True,#是否截断超长序列	
                            add_special_tokens=False,#需要原始词表对齐的任务
                            max_length=512,#控制输入大小
                            is_split_into_words=True,
                            return_offsets_mapping=True,
                            ) #输入是否已分词	
    labels= [lbl[:512] for lbl in item['ner_tags']]
    input_data['labels'] =labels
    return input_data

ds1 =ds.map(data_input_proc,batched=True)



In [ ]:
ds1.set_format('torch',columns=['input_ids','token_type_ids','attention_mask','labels'])

In [ ]:
id2label = {i:tag for i,tag in enumerate(tags)}
label2id = {tag:i for i,tag in enumerate(tags)}
model=AutoModelForTokenClassification.from_pretrained('bert-base-chinese',
                                                      num_labels = len(tags),
                                                      id2label = id2label,
                                                      label2id = label2id)
model



In [ ]:
args = TrainingArguments(
    output_dir="msra_ner_train",
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    report_to="tensorboard",
    eval_strategy="epoch"
)

In [ ]:
def compute_metrics(result):
    #result 是元组-tule(predocts,lables)，

    #获取评估对象
    seqeval=evaluate.load('seqeval')
    predicts,labels= result
    predicts=np.argmax(predicts,axis=2)
    
    #准备评估数据
    predicts=[[tags[p] for p,l in zip(ps,ls) if l!=-100]
              for ps,ls in zip(predicts,labels)]
    labels=[[tags[l] for p,l in zip(ps,ls) if l!=-100]
              for ps,ls in zip(predicts,labels)]

    results =seqeval.compute(predictions=predicts,references=labels) 

    return results

In [ ]:
data_collator= DataCollatorForTokenClassification(tokenizer=tokenizer,padding=True)

In [ ]:
trainer= Trainer(
    model,
    args,
    train_dataset=ds1['train'],
    eval_dataset=ds1['test'],
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

In [ ]:
trainer.train()

In [ ]:
from transformers import pipeline

In [ ]:
pipeline = pipeline('token-classification','msra_ner_train/checkpoint-4221')

In [ ]:
pipeline('双方确定了今后发展中美关系的指导方针')